In [4]:
import os
seed = 0
os.environ['PYTHONHASSEED'] = str(seed)

In [5]:
# Disable GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [6]:
import h5py
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import time
import random
import pickle

from tensorflow.random import set_seed

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

In [7]:
# Set seed
random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [8]:
filename = '../data/turbofan_dataset/N-CMAPSS_DS02-006.h5'
output_path = 'DS02/experiment_set_6'

In [9]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [10]:
# Constants
TEST_SET_SIZE = 0.3

In [11]:
def load_dataset(filename, load_test_set=True):
    """ Reads a dataset from a given .h5 file and compose (in memory) the train and test data. 
    Args:
        filename(str): path to the .h5 file
    Returns:
        train_set(pd.DataFrame), test_set(pd.DataFrame)
    """
    with h5py.File(filename, 'r') as hdf:
        # Development set
        W_dev = np.array(hdf.get('W_dev'))             # W
        X_s_dev = np.array(hdf.get('X_s_dev'))         # X_s
        X_v_dev = np.array(hdf.get('X_v_dev'))         # X_v
        T_dev = np.array(hdf.get('T_dev'))             # T
        Y_dev = np.array(hdf.get('Y_dev'))             # RUL  
        A_dev = np.array(hdf.get('A_dev'))             # Auxiliary

        # Test set
        if load_test_set:
            W_test = np.array(hdf.get('W_test'))           # W
            X_s_test = np.array(hdf.get('X_s_test'))       # X_s
            X_v_test = np.array(hdf.get('X_v_test'))       # X_v
            T_test = np.array(hdf.get('T_test'))           # T
            Y_test = np.array(hdf.get('Y_test'))           # RUL  
            A_test = np.array(hdf.get('A_test'))           # Auxiliary
        
        # Column names
        W_var = np.array(hdf.get('W_var'))
        X_s_var = np.array(hdf.get('X_s_var'))  
        X_v_var = np.array(hdf.get('X_v_var')) 
        T_var = np.array(hdf.get('T_var'))
        A_var = np.array(hdf.get('A_var'))
        
        columns = []
        columns.append(list(np.array(A_var, dtype='U20')))
        columns.append(list(np.array(T_var, dtype='U20')))
        columns.append(list(np.array(X_s_var, dtype='U20')))
        columns.append(list(np.array(X_v_var, dtype='U20')))
        columns.append(list(np.array(W_var, dtype='U20')))
        columns.append(['RUL'])
        
        columns_list = []
        for columns_per_category in columns:
            columns_list += columns_per_category
        
    train_set = np.concatenate((A_dev, T_dev, X_s_dev, X_v_dev, W_dev, Y_dev), axis=1)
    if load_test_set:
        test_set = np.concatenate((A_test, T_test, X_s_test, X_v_test, W_test, Y_test), axis=1)
        return pd.DataFrame(data=train_set, columns=columns_list), pd.DataFrame(data=test_set, columns=columns_list), columns
    else:
        return pd.DataFrame(data=train_set, columns=columns_list), None, columns

In [12]:
def unit_cycle_info(df, compute_cycle_len=False):
    unit_ids = np.unique(df['unit'])
    print('Engine units in df: ', unit_ids)
    for i in unit_ids:
        num_cycles = len(np.unique(df.loc[df['unit'] == i, 'cycle']))
        print('Unit: ', i, ' - Number of flight cycles: ', num_cycles)
        
    if compute_cycle_len:
        cycle_ids = np.unique(df['cycle'])
        print('Total number of cycles: ', len(cycle_ids))
        min_len = np.inf
        max_len = 0
        for i in cycle_ids:
            cycle_len = len(df.loc[df['cycle'] == i, 'cycle'])
            if cycle_len < min_len:
                min_len = cycle_len
            elif cycle_len > max_len:
                max_len = cycle_len
        print('Min cycle length: ', min_len)
        print('Max cycle length: ', max_len)
    
    return unit_ids

def get_quasi_constant_features(dataset, variance_th=0.01, debug=True):
    constant_filter = VarianceThreshold(threshold=variance_th)
    constant_filter.fit(dataset)
    constant_features = [col for col in dataset.columns 
                         if col not in dataset.columns[constant_filter.get_support()]]
    
    if debug:
        print("Number of non-constant features: ", len(dataset.columns[constant_filter.get_support()]))
        
        print("Number of quasi-constant features: ", len(constant_features))
        print("Quasi-constant features: ")
        for col in constant_features:
            print(col)
    return constant_features

def get_principal_components(X, explained_variance=0.99, debug=False):   
    pca = PCA(explained_variance, svd_solver='full', random_state=seed)
    pca = pca.fit(X)
    
    if debug:
        explained_variance_cumsum = np.cumsum(pca.explained_variance_ratio_)
        pca_dims = np.argmax(explained_variance_cumsum >= explained_variance) + 1
        print(f"Can reduce from {X.shape[1]} to {pca_dims} dimensions while retaining {explained_variance}% of variance.")

        plt.plot(explained_variance_cumsum)
        plt.xlabel('Number of components')
        plt.ylabel('Cumulative explained variance')

        plt.figure()
        plt.bar(range(pca.components_.shape[0]), pca.explained_variance_ratio_)
        plt.xlabel('PC')
        plt.ylabel('Explained variance')
    return pca
    
def dimensionality_reduction(X, pca):
    return pca.transform(X)

def cmapss_score_function(actual, predictions, normalize=True):
    # diff < 0 -> over-estimation
    # diff > 0 -> under-estimation
    diff = actual - predictions
    alpha = np.full_like(diff, 1/13)
    negative_diff_mask = diff < 0
    alpha[negative_diff_mask] = 1/10
    score = np.sum(np.exp(alpha * np.abs(diff)))
    
    if normalize:
        N = len(predictions)
        score /= N
    return score

def compute_evaluation_metrics(actual, predictions, label='Test'):
    mse = mean_squared_error(actual, predictions)
    rmse = np.sqrt(mse)
    cmapss_score = cmapss_score_function(actual, predictions)
    print('{} set:\nMSE: {:.2f}\nRMSE: {:.2f}\nCMAPSS score: {:.2f}\n'.format(label, mse, rmse, 
                                                                     cmapss_score))
    return mse, rmse, cmapss_score
    
def plot_loss_curves(history, output_path=None, y_lim=[0, 150]):
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.ylim(y_lim)
    plt.legend(['train', 'validation'], loc='upper left')
    
    if output_path is not None:
        plt.savefig(os.path.join(output_path, 'loss_curves.png'), format='png', dpi=300) 
    plt.show()
    
def plot_rul(expected, predicted):
    plt.figure()
    plt.plot(range(len(expected)), expected, label='Expected')
    plt.plot(range(len(predicted)), predicted, label='Predicted')
    plt.legend()
    
    
def create_mlp_model(input_dim, hidden_layer_sizes, activation='relu', output_weights_file=None):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], 
                    input_dim=input_dim, 
                    kernel_initializer='random_normal', 
                    activation=activation))

    for layer_size in hidden_layer_sizes[1:]:
        model.add(Dense(layer_size, 
                        kernel_initializer='random_normal', 
                        activation=activation))
    
    model.add(Dense(1, kernel_initializer='random_normal'))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    if output_weights_file is not None:
        model.save_weights(output_weights_file)
    return model

def train_model_existing_weights(model, weights_file, x_train, y_train, x_val, y_val, epochs=200, batch_size=512, callbacks=[]):
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.load_weights(weights_file)
    return model.fit(x_train, y_train,
                     validation_data=(x_val, y_val),
                     epochs=epochs,
                     batch_size=batch_size,
                     verbose=1,
                     callbacks=callbacks)

def save_history(history, output_file=os.path.join(output_path, "history.pkl")):
    with open(output_file, 'wb') as file:
        pickle.dump(history.history, file)
    print("Saved training history to file: {}".format(output_file))

def load_history(file):
    return pickle.load(open(file, "rb"))

def save_object(obj, output_file):
    with open(output_file, 'wb') as file:
        pickle.dump(obj, file)
    print("Saved object to file: {}".format(output_file))
    
def load_object(file):
    return pickle.load(open(file, "rb"))

def model_evaluation(model, x_test, y_test, x_train=None, y_train=None, plot_range=[0, 10**3]):
    if x_train is not None and y_train is not None:
        predictions_train = model.predict(x_train).flatten()
        compute_evaluation_metrics(predictions_train, y_train, 'Train')
        
        expected = y_train[plot_range[0]:plot_range[1]]
        predicted = predictions_train[plot_range[0]:plot_range[1]]
        plot_rul(expected, predicted)
        
    predictions_test = model.predict(x_test).flatten()
    compute_evaluation_metrics(predictions_test, y_test)
    
    expected = y_test[plot_range[0]:plot_range[1]]
    predicted = predictions_test[plot_range[0]:plot_range[1]]
    plot_rul(expected, predicted)

def numbers_list_to_string(num_list):
    return " ".join([str(x) for x in num_list])

In [13]:
start_time = time.process_time()  
train_set, test_set, columns = load_dataset(filename)
print("Operation time (sec): " , (time.process_time() - start_time))
print()
print("Train set shape: " + str(train_set.shape))

columns_aux = columns[0] 
columns_health_params = columns[1] 
columns_sensor_measurements = columns[2] 
columns_virtual_sensors = columns[3]
columns_operating_conditions = columns[4] 
target_col = columns[5]

Operation time (sec):  3.90625

Train set shape: (5263447, 47)


In [14]:
y_train = train_set['RUL']
x_train = train_set.drop(['RUL'], axis=1)

In [15]:
constant_features = get_quasi_constant_features(x_train, variance_th=0)
x_train.drop(labels=constant_features, axis=1, inplace=True)
print("Train shape: ", x_train.shape)

Number of non-constant features:  38
Number of quasi-constant features:  8
Quasi-constant features: 
Fc
fan_eff_mod
fan_flow_mod
LPC_eff_mod
LPC_flow_mod
HPC_eff_mod
HPC_flow_mod
HPT_flow_mod
Train shape:  (5263447, 38)


In [16]:
# Drop auxiliary data columns
x_train.drop(labels=[x for x in columns_aux if x in x_train.columns], axis=1, inplace=True)

In [17]:
y_test = test_set['RUL']
x_test = test_set[x_train.columns]

In [19]:
###########################################
# Test effect of PCA for feature extraction
###########################################
NUM_TRIALS = 5

batch_size = 512
epochs = 200
layer_sizes = [256, 256, 512, 64]

results_path = os.path.join(output_path, "results_pca")
results_file = os.path.join(results_path, "results_pca.csv")
with open(results_file, "w") as file:
    file.write("mse,rmse,cmapss,mse(mean),mse(std),rmse(mean),rmse(std),cmapss(mean),cmapss(std)\n")

mse_vals = []
rmse_vals = []
cmapss_vals = []
    
for random_seed in range(NUM_TRIALS):
    # Train-validation split for early stopping
    x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train, 
                                                                              y_train, 
                                                                              test_size=0.3, 
                                                                              random_state=random_seed)
    # Create output path
    results_path_crr_split = os.path.join(results_path, "split_{}".format(random_seed))
    if not os.path.exists(results_path_crr_split):
        os.makedirs(results_path_crr_split)

    # Standardization
    scaler_file = os.path.join(results_path_crr_split, 'scaler.pkl')
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train_split)
    x_val_scaled = scaler.transform(x_val_split)
    x_test_scaled = scaler.transform(x_test)
    save_object(scaler, scaler_file)
    
    # PCA
    pca = get_principal_components(x_train_scaled, debug=False)
    pca_file = os.path.join(results_path_crr_split, 'pca.pkl')
    save_object(pca, pca_file)
    
    x_train_final = dimensionality_reduction(x_train_scaled, pca)
    x_val_final = dimensionality_reduction(x_val_scaled, pca)
    x_test_final = dimensionality_reduction(x_test_scaled, pca)
    input_dim = x_train_final.shape[1]

    # Create model
    weights_file = os.path.join(results_path, 'mlp_initial_weights.h5')
    model_path = os.path.join(results_path_crr_split, 'mlp_model_trained.h5')
        
    # Save initial weights
    if random_seed == 0:
        model = create_mlp_model(input_dim, layer_sizes, activation='tanh',
                                  output_weights_file=weights_file)
    else:
        model = create_mlp_model(input_dim, layer_sizes, activation='tanh')
    model.summary()

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    mc = ModelCheckpoint(model_path, monitor='val_loss', mode='min', verbose=2, 
                         save_best_only=True)

    # Train model
    history = train_model_existing_weights(model, weights_file, 
                                           x_train_final, y_train_split, 
                                           x_val_final, y_val_split, 
                                           batch_size=batch_size, 
                                           epochs=epochs, 
                                           callbacks=[es, mc])

    history_file = os.path.join(results_path_crr_split, "history.pkl")
    save_history(history, history_file)

    # Performance evaluation
    loaded_model = load_model(model_path)
    predictions_test = loaded_model.predict(x_test_final).flatten()
    mse, rmse, cmapss_score = compute_evaluation_metrics(predictions_test, y_test)
        
    mse_vals.append(mse)
    rmse_vals.append(rmse)
    cmapss_vals.append(cmapss_score)
    
mse_mean = np.mean(mse_vals)
mse_std = np.std(mse_vals)
rmse_mean = np.mean(rmse_vals)
rmse_std = np.std(rmse_vals)
cmapss_mean = np.mean(cmapss_vals)
cmapss_std = np.std(cmapss_vals)
    
with open(results_file, "a") as file:   
    file.write(f"{numbers_list_to_string(mse_vals)}, {numbers_list_to_string(rmse_vals)}, {numbers_list_to_string(cmapss_vals)}, {mse_mean}, {mse_std}, {rmse_mean}, {rmse_std}, {cmapss_mean}, {cmapss_std}\n")

Saved object to file: DS02/experiment_set_6\results_pca\split_0\scaler.pkl
Saved object to file: DS02/experiment_set_6\results_pca\split_0\pca.pkl
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               1792      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 232,065
Trainable params: 232,065
Non-trainable params: 0
___________________________________

7197/7197 [==============================] - 38s 5ms/step - loss: 1.9695 - val_loss: 1.9302
Epoch 27/200
7187/7197 [============================>.] - ETA: 0s - loss: 1.9419
Epoch 00027: val_loss improved from 1.93020 to 1.89174, saving model to DS02/experiment_set_6\results_pca\split_0\mlp_model_trained.h5
7197/7197 [==============================] - 36s 5ms/step - loss: 1.9417 - val_loss: 1.8917
Epoch 28/200
7186/7197 [============================>.] - ETA: 0s - loss: 1.9227
Epoch 00028: val_loss did not improve from 1.89174
7197/7197 [==============================] - 36s 5ms/step - loss: 1.9220 - val_loss: 1.9245
Epoch 29/200
7195/7197 [============================>.] - ETA: 0s - loss: 1.9075
Epoch 00029: val_loss did not improve from 1.89174
7197/7197 [==============================] - 35s 5ms/step - loss: 1.9076 - val_loss: 2.0436
Epoch 30/200
7194/7197 [============================>.] - ETA: 0s - loss: 1.8676
Epoch 00030: val_loss did not improve from 1.89174
7197/7197 [=========

7197/7197 [==============================] - ETA: 0s - loss: 1.4794
Epoch 00059: val_loss did not improve from 1.41848
7197/7197 [==============================] - 35s 5ms/step - loss: 1.4794 - val_loss: 1.5750
Epoch 60/200
7188/7197 [============================>.] - ETA: 0s - loss: 1.4709
Epoch 00060: val_loss improved from 1.41848 to 1.29716, saving model to DS02/experiment_set_6\results_pca\split_0\mlp_model_trained.h5
7197/7197 [==============================] - 37s 5ms/step - loss: 1.4706 - val_loss: 1.2972
Epoch 61/200
7196/7197 [============================>.] - ETA: 0s - loss: 1.4614
Epoch 00061: val_loss did not improve from 1.29716
7197/7197 [==============================] - 35s 5ms/step - loss: 1.4614 - val_loss: 1.6744
Epoch 62/200
7185/7197 [============================>.] - ETA: 0s - loss: 1.4489
Epoch 00062: val_loss did not improve from 1.29716
7197/7197 [==============================] - 35s 5ms/step - loss: 1.4494 - val_loss: 1.7070
Epoch 63/200
7192/7197 [=========

7186/7197 [============================>.] - ETA: 0s - loss: 3.4169
Epoch 00009: val_loss did not improve from 3.42666
7197/7197 [==============================] - 36s 5ms/step - loss: 3.4166 - val_loss: 3.4978
Epoch 10/200
7184/7197 [============================>.] - ETA: 0s - loss: 3.2106
Epoch 00010: val_loss did not improve from 3.42666
7197/7197 [==============================] - 36s 5ms/step - loss: 3.2104 - val_loss: 5.7608
Epoch 11/200
7196/7197 [============================>.] - ETA: 0s - loss: 3.0787
Epoch 00011: val_loss improved from 3.42666 to 3.28600, saving model to DS02/experiment_set_6\results_pca\split_1\mlp_model_trained.h5
7197/7197 [==============================] - 36s 5ms/step - loss: 3.0787 - val_loss: 3.2860
Epoch 12/200
7190/7197 [============================>.] - ETA: 0s - loss: 2.9563
Epoch 00012: val_loss improved from 3.28600 to 3.25010, saving model to DS02/experiment_set_6\results_pca\split_1\mlp_model_trained.h5
7197/7197 [==============================

Epoch 42/200
7194/7197 [============================>.] - ETA: 0s - loss: 1.7130
Epoch 00042: val_loss improved from 1.58039 to 1.51682, saving model to DS02/experiment_set_6\results_pca\split_1\mlp_model_trained.h5
7197/7197 [==============================] - 33s 5ms/step - loss: 1.7129 - val_loss: 1.5168
Epoch 43/200
7187/7197 [============================>.] - ETA: 0s - loss: 1.6644- ETA: 0s
Epoch 00043: val_loss did not improve from 1.51682
7197/7197 [==============================] - 33s 5ms/step - loss: 1.6647 - val_loss: 1.9957
Epoch 44/200
7193/7197 [============================>.] - ETA: 0s - loss: 1.6510
Epoch 00044: val_loss did not improve from 1.51682
7197/7197 [==============================] - 33s 5ms/step - loss: 1.6510 - val_loss: 1.8899
Epoch 45/200
7190/7197 [============================>.] - ETA: 0s - loss: 1.6474
Epoch 00045: val_loss did not improve from 1.51682
7197/7197 [==============================] - 33s 5ms/step - loss: 1.6471 - val_loss: 1.8664
Epoch 46/20

...

7196/7197 [============================>.] - ETA: 0s - loss: 1.7820
Epoch 00037: val_loss did not improve from 1.78422
7197/7197 [==============================] - 35s 5ms/step - loss: 1.7821 - val_loss: 1.9467
Epoch 38/200
7184/7197 [============================>.] - ETA: 0s - loss: 1.7413
Epoch 00038: val_loss did not improve from 1.78422
7197/7197 [==============================] - 35s 5ms/step - loss: 1.7411 - val_loss: 2.0081
Epoch 39/200
7192/7197 [============================>.] - ETA: 0s - loss: 1.7286
Epoch 00039: val_loss improved from 1.78422 to 1.53895, saving model to DS02/experiment_set_6\results_pca\split_2\mlp_model_trained.h5
7197/7197 [==============================] - 36s 5ms/step - loss: 1.7286 - val_loss: 1.5390
Epoch 40/200
7189/7197 [============================>.] - ETA: 0s - loss: 1.7140
Epoch 00040: val_loss did not improve from 1.53895
7197/7197 [==============================] - 35s 5ms/step - loss: 1.7136 - val_loss: 1.8448
Epoch 41/200
7191/7197 [=========

Saved object to file: DS02/experiment_set_6\results_pca\split_3\pca.pkl
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 256)               1792      
_________________________________________________________________
dense_16 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_17 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_18 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 65        
Total params: 232,065
Trainable params: 232,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
7186/7197 [===================

7185/7197 [============================>.] - ETA: 0s - loss: 1.9689- ETA: 0s 
Epoch 00026: val_loss improved from 1.96064 to 1.82508, saving model to DS02/experiment_set_6\results_pca\split_3\mlp_model_trained.h5
7197/7197 [==============================] - 36s 5ms/step - loss: 1.9684 - val_loss: 1.8251
Epoch 27/200
7188/7197 [============================>.] - ETA: 0s - loss: 1.9325
Epoch 00027: val_loss did not improve from 1.82508
7197/7197 [==============================] - 35s 5ms/step - loss: 1.9325 - val_loss: 1.9621
Epoch 28/200
7186/7197 [============================>.] - ETA: 0s - loss: 1.9154
Epoch 00028: val_loss did not improve from 1.82508
7197/7197 [==============================] - 35s 5ms/step - loss: 1.9151 - val_loss: 2.0261
Epoch 29/200
7193/7197 [============================>.] - ETA: 0s - loss: 1.9085
Epoch 00029: val_loss did not improve from 1.82508
7197/7197 [==============================] - 35s 5ms/step - loss: 1.9089 - val_loss: 1.8493
Epoch 30/200
7195/7197 

7193/7197 [============================>.] - ETA: 0s - loss: 2.6982
Epoch 00012: val_loss did not improve from 2.56991
7197/7197 [==============================] - 36s 5ms/step - loss: 2.6979 - val_loss: 2.6942
Epoch 13/200
7194/7197 [============================>.] - ETA: 0s - loss: 2.5879
Epoch 00013: val_loss did not improve from 2.56991
7197/7197 [==============================] - 36s 5ms/step - loss: 2.5882 - val_loss: 2.6061
Epoch 14/200
7192/7197 [============================>.] - ETA: 0s - loss: 2.4779
Epoch 00014: val_loss improved from 2.56991 to 2.56137, saving model to DS02/experiment_set_6\results_pca\split_4\mlp_model_trained.h5
7197/7197 [==============================] - 36s 5ms/step - loss: 2.4778 - val_loss: 2.5614
Epoch 15/200
7191/7197 [============================>.] - ETA: 0s - loss: 2.4391
Epoch 00015: val_loss did not improve from 2.56137
7197/7197 [==============================] - 36s 5ms/step - loss: 2.4392 - val_loss: 2.7317
Epoch 16/200
7189/7197 [=========

7192/7197 [============================>.] - ETA: 0s - loss: 1.6228
Epoch 00045: val_loss did not improve from 1.56316
7197/7197 [==============================] - 34s 5ms/step - loss: 1.6228 - val_loss: 1.9158
Epoch 46/200
7186/7197 [============================>.] - ETA: 0s - loss: 1.5886
Epoch 00046: val_loss did not improve from 1.56316
7197/7197 [==============================] - 34s 5ms/step - loss: 1.5886 - val_loss: 1.5728
Epoch 47/200
7197/7197 [==============================] - ETA: 0s - loss: 1.6128
Epoch 00047: val_loss did not improve from 1.56316
7197/7197 [==============================] - 34s 5ms/step - loss: 1.6128 - val_loss: 1.6792
Epoch 48/200
7183/7197 [============================>.] - ETA: 0s - loss: 1.5605
Epoch 00048: val_loss improved from 1.56316 to 1.48890, saving model to DS02/experiment_set_6\results_pca\split_4\mlp_model_trained.h5
7197/7197 [==============================] - 34s 5ms/step - loss: 1.5599 - val_loss: 1.4889
Epoch 49/200
7191/7197 [=========